## Modules

In [1]:
# Modules
import sys
sys.path.append("/home/simon/Documents/REPO/Exact_laws")
sys.path.append("/home/simon/Documents/REPO/Simu/Rapport_simu_exactlaw/notebook")

from exact_laws.preprocessing.process_on_standard_h5_file import describ_file
from visualisation_tools import *
from exact_laws.mathematical_tools.derivation import cdiff

import os
from matplotlib import ticker, rc
from matplotlib import gridspec
import numpy as np
import json

In [2]:
# Matplotlib config
import matplotlib
%matplotlib qt5
plt.rcParams['text.usetex'] = True
matplotlib.rc('text.latex', preamble=r'\usepackage{amsmath}')
rc('xtick', labelsize=15) 
rc('ytick', labelsize=15) 
rc('font', size=18)

numfig = 0

## Informations et données

In [3]:
simu_name = 'CGL5'
with open("/home/simon/Documents/REPO/Simu/descript_oca_simu.json","r") as f:
    simus = json.load(f)[simu_name]
#del(simus['10600'],simus['12500'],simus['12785'],simus['12800'],simus['12815'],)
times = list(simus.keys())
for t in times : 
    if not '12905.' in t: del(simus[t])
times = list(simus.keys())
simus_name= 'CGL5.12905'
print(simus[times[0]])

{'original_folder': '/home/simon/Documents/DATA/SIMULATIONS/OCA/data_simus_oca/CGL5/CGL5_12905/', 'cycle': 'cycle_0', 'group_grid': '3Dgrid', 'N': [512, 512, 1024], 'kdi': 0.14705882352941177, 'theta': 1.309017176581171, 'thrsh_forc_hi': 0.016, 'thrsh_forc_lo': 0.015, 'a1_forc': 0.5, 'act_forc': True, 'nu': 3e-11, 'eta': 3e-11, 'hdiss_rho': 0, 'hdiss_p': 3e-12, 'an_hd': 6, 'ap0': 1, 'B0': [0, 0, 1], 'beta0': 1, 'name': 'CGL5.12905.000', 'EL_red1D': ['/home/simon/Documents/DATA/SIMULATIONS/OCA/runs/CGL5.12905.0/OCA_CGL5.12905.0_alli_EL_lincart_all_red1D.h5', '/home/simon/Documents/DATA/SIMULATIONS/OCA/runs/CGL5.12905.0/OCA_CGL5.12905.0_alleq_EL_lincart_all_red1D.h5', '/home/simon/Documents/DATA/SIMULATIONS/OCA/runs/CGL5.12905.0/OCA_CGL5.12905.0_allkhm_EL_lincart_all_red1D.h5'], 'EL_red2D': ['/home/simon/Documents/DATA/SIMULATIONS/OCA/runs/CGL5.12905.0/OCA_CGL5.12905.0_alli_EL_lincart_all_red2D.h5', '/home/simon/Documents/DATA/SIMULATIONS/OCA/runs/CGL5.12905.0/OCA_CGL5.12905.0_alleq_EL_l

In [4]:
# Paths 
path_record_images = f"./images_{simu_name}/"
if not os.path.exists(path_record_images) : os.mkdir(path_record_images)

In [5]:
# Paramètres
simu = f"{simu_name}"
kdi = simus[times[0]]['kdi'] # kperpdi = 2pidi/l => l/di = 2pi/(kdi)
theta = simus[times[0]]['theta']
lforc = np.array([(2*np.pi/kdi, 2*np.pi/kdi*np.tan(theta)),(2*np.pi/kdi/np.sqrt(2), 2*np.pi/kdi*np.tan(theta))]) # (lperp/di, lpar/di)
linert = (1,1)
print(lforc)

[[ 42.72566009 159.4672431 ]
 [ 30.21160398 159.4672431 ]]


In [6]:
# Grille
grid = read_grid_in_h5file(simus[times[0]]['EL_red2D'][0])
c = grid['c']
axe_par = np.array(grid['lpar'])*grid['c'][2]
axe_perp = np.array(grid['lperp'])#*grid['c'][0]
label_par = "$\ell_{\parallel} / d_i$"
label_perp = "$\ell_{\perp}/ d_i$"
del(grid)
grid = read_grid_in_h5file(simus[times[0]]['EL_red1D'][0])
axe_l = np.array(grid['l'])
label_l = "$\ell / d_i$"
print(axe_par[1],axe_par[2],max(axe_par))
print(axe_perp[1],axe_perp[2],max(axe_perp))
print(axe_l[1],max(axe_l))

0.15572972958615028 0.31145945917230056 79.57789181852279
0.08344855486097888 0.16689710972195776 21.279381489549614
0.08344855486097888 21.279381489549614


In [7]:
gridlp, gridlpar = np.meshgrid(axe_perp,axe_par, indexing='ij')
filt_45 = np.arctan(gridlpar/np.where(gridlp==0,1e-19,gridlp))*180/np.pi
filt_45inf = np.where(filt_45<=45,1,np.nan)
filt_45sup = np.where(filt_45>=45,1,np.nan)
filt_45b = np.arctan(gridlpar/np.where(gridlp==0,1e-19,gridlp))
filt_45binf = np.where(filt_45b<=theta,1,np.nan)
filt_45bsup = np.where(filt_45b>=theta,1,np.nan)
filt_perp10pc = np.where(gridlp<=lforc[1,0]*0.1,1,np.nan)
filt_par10pc = np.where(gridlpar<=lforc[1,1]*0.1,1,np.nan)
del(gridlp,gridlpar)

In [8]:
# liste des temps
dict_temps = {}
for t in times : 
    if '_R' in t : 
        dict_temps[t] = eval(t[:-2])
    else : 
        dict_temps[t] = eval(t)
temps =  sorted(dict_temps.items(), key=lambda t: t[1])
dt = temps[1][1] - temps[0][1]

## Lecture et assemblage de lois exactes

In [9]:
# Lecture fichiers et corrections
dict_quantities = {}
dict_quantities1D = {}
dict_coeffs = {}
quantities = {} 
quantities1D = {}  
coeffs = {}

for it,t in enumerate(simus.keys()):
    dict_quantities[t] = {}
    dict_coeffs[t] = {}
    for fstr in simus[t]['EL_red2D']:
        dqs, _, dcs = readfile(fstr)
        for k in dqs.keys():
            dict_quantities[t][k] = dqs[k] 
        for k in dcs.keys():
            dict_coeffs[t][k] = dcs[k]
            if "source_rbdd" in k:
                dict_coeffs[t][k] = -np.abs(dcs[k])
            if "ue" in k and not 'KHM' in k :
                dict_coeffs[t][k] = -np.abs(dcs[k])*1836
    dict_coeffs[t]['COR_dt_cor_rvv'] = 1
    dict_coeffs[t]['COR_dt_cor_rbb'] = 1
    dict_coeffs[t]['COR_dt_cor_ru'] = 1
    dict_coeffs[t]['COR_dt_cor_rue'] = 1836
    dict_coeffs[t]['ICOR_dt_cor_vv'] = 1
    dict_coeffs[t]['ICOR_dt_cor_bb'] = 1
    
    dict_quantities1D[t] = {}
    for fstr in simus[t]['EL_red1D']:
        dqs, _, dcs = readfile(fstr)
        for k in dqs.keys():
            dict_quantities1D[t][k] = dqs[k] 
for q in dict_quantities[times[-1]].keys():
    quantities[q] = np.mean(np.array([dict_quantities[t][q] for t in simus.keys() if q in dict_quantities[t].keys()]),axis=0)
for q in dict_quantities1D[times[-1]].keys():
    quantities1D[q] = np.mean(np.array([dict_quantities1D[t][q] for t in simus.keys() if q in dict_quantities[t].keys()]),axis=0)
for q in dict_coeffs[times[-1]].keys():
    coeffs[q] = dict_coeffs[times[-1]][q]


In [10]:
# dt des quantitées
quantities_name = [k for k in quantities if k.startswith('cor_')]
for q in quantities_name:
    tab = []
    for t in temps: 
        tab.append(dict_quantities[t[0]][q])
    dt_tab = cdiff(tab, length_case=dt, dirr=0, precision=2, period=False, point=False)
    for i in range(len(temps)):
        dict_quantities[temps[i][0]]['dt_'+q] = dt_tab[i]
    quantities['dt_'+q] = np.mean(dt_tab[1:-1],axis=0)

# dt des quantitées
quantities_name = [k for k in quantities1D if k.startswith('cor_')]
for q in quantities_name:
    tab = []
    for t in temps: 
        tab.append(dict_quantities1D[t[0]][q])
    dt_tab = cdiff(tab, length_case=dt, dirr=0, precision=2, period=False, point=False)
    if q == 'cor_rvv': print(dt_tab[0][0],dt_tab[1][0],dt_tab[1][0])
    for i in range(len(temps)):
        dict_quantities1D[temps[i][0]]['dt_'+q] = dt_tab[i]
    quantities1D['dt_'+q] = np.mean(dt_tab,axis=0)

1.978280450430888e-05 1.97553844654088e-05 1.97553844654088e-05


In [11]:
# Combinaisons de termes
dic_of_list_terms = {}
dic_of_list_terms['ELcgl'] = [k for k in coeffs if k.startswith('SS22Gyr')]
dic_of_list_terms['ELiso'] = [k for k in coeffs if k.startswith('SS22Iso')]
print(dic_of_list_terms['ELiso'])
dic_of_list_terms['ELelec']  =  ['SS22Elec_div_flux_drduedv','SS22Elec_div_flux_drdpedv',
                               'SS22Elec_source_pevdrdr','SS22Elec_source_rdpedv',
                               'SS22Elec_source_rduedv','SS22Elec_source_rvdpedr']
dic_of_list_terms['ELhall'] = [k for k in coeffs if k.startswith('Hallcor')]
dic_of_list_terms['EL'] = dic_of_list_terms['ELcgl'] + dic_of_list_terms['ELelec'] + dic_of_list_terms['ELhall']
dic_of_list_terms['ELIso'] = dic_of_list_terms['ELiso'] + dic_of_list_terms['ELelec'] + dic_of_list_terms['ELhall']
dic_of_list_terms['dtR'] = [k for k in coeffs if k.startswith('COR_dt')]
dic_of_list_terms['khm'] = [k for k in coeffs if k.startswith('SS22KHM')]
dic_of_list_terms['ELcgl_flux'] = ['SS22Gyr_div_flux_drdpperpdv','SS22Gyr_div_flux_drdpandv',]
dic_of_list_terms['ELcgl_sdv'] = ['SS22Gyr_source_rdpandv', 'SS22Gyr_source_rdpperpdv',]
dic_of_list_terms['ELcgl_sdp'] = ['SS22Gyr_source_rvdpperpdr','SS22Gyr_source_rvdpandr',]
dic_of_list_terms['ELcgl_sdr'] = ['SS22Gyr_source_panvdrdr','SS22Gyr_source_pperpvdrdr',]
dic_of_list_terms['ELiso_flux'] = ['SS22Iso_div_flux_drdpisodv',]
dic_of_list_terms['ELiso_sdv'] = ['SS22Iso_source_rdpisodv',]
dic_of_list_terms['ELiso_sdp'] = ['SS22Iso_source_rvdpisodr',]
dic_of_list_terms['ELiso_sdr'] = ['SS22Iso_source_pisovdrdr',]

['SS22Iso_div_flux_drbdbdv', 'SS22Iso_div_flux_drbdvdb', 'SS22Iso_div_flux_drdpisodv', 'SS22Iso_div_flux_drdpmdv', 'SS22Iso_div_flux_drduisodv', 'SS22Iso_div_flux_drvdbdb', 'SS22Iso_div_flux_drvdvdv', 'SS22Iso_source_bdrbdv', 'SS22Iso_source_bdrvdb', 'SS22Iso_source_pisovdrdr', 'SS22Iso_source_pmvdrdr', 'SS22Iso_source_rbdbdv', 'SS22Iso_source_rbdvdb', 'SS22Iso_source_rdpisodv', 'SS22Iso_source_rduisodv', 'SS22Iso_source_rvdbdb', 'SS22Iso_source_rvdpisodr', 'SS22Iso_source_rvdpmdr', 'SS22Iso_source_rvdvdv']


In [12]:
zeta = (-linear_op_from_list_term(coeffs,quantities,dic_of_list_terms['dtR']) 
        + linear_op_from_list_term(coeffs,quantities,dic_of_list_terms['khm']) 
        -linear_op_from_list_term(coeffs,quantities,dic_of_list_terms['EL']))

In [13]:
plt.figure('M2', figsize=(8,4))
plt.clf()

y1 = linear_op_from_list_term(coeffs,quantities,dic_of_list_terms['EL'])
y2 = linear_op_from_list_term(coeffs,quantities,dic_of_list_terms['ELIso'])
splot(axe_perp[1:],np.nanmean((y1-y2)*filt_45binf,axis=1)[1:],label = '$\\varepsilon_{\overline{\\boldsymbol{\\Pi}}}$') 

splot(axe_perp[1:],np.nanmean(np.abs(zeta)*filt_45binf,axis=1)[1:],label = '$\\zeta$',color = 'grey')
plt.fill_between(axe_perp[1:],np.nanmean(np.abs(zeta)*filt_45binf,axis=1)[1:],color = 'grey', alpha = 0.3)

y1 = linear_op_from_list_term(coeffs,quantities,dic_of_list_terms['ELcgl_flux'])
y2 = linear_op_from_list_term(coeffs,quantities,dic_of_list_terms['ELiso_flux'])
splot(axe_perp[1:],np.nanmean((y1-y2)*filt_45binf,axis=1)[1:],label = '$\\nabla_{\\ell} \cdot \\mathbf{\mathcal{F}}_{A}$') 

y1 = linear_op_from_list_term(coeffs,quantities,dic_of_list_terms['ELcgl_sdv'])
y2 = linear_op_from_list_term(coeffs,quantities,dic_of_list_terms['ELiso_sdv'])
splot(axe_perp[1:],np.nanmean((y1-y2)*filt_45binf,axis=1)[1:],label = '$\mathcal{S}_{A1}$') 

y1 = linear_op_from_list_term(coeffs,quantities,dic_of_list_terms['ELcgl_sdp'])
y2 = linear_op_from_list_term(coeffs,quantities,dic_of_list_terms['ELiso_sdp'])
splot(axe_perp[1:],np.nanmean((y1-y2)*filt_45binf,axis=1)[1:],label = '$\mathcal{S}_{A2}$') 

y1 = linear_op_from_list_term(coeffs,quantities,dic_of_list_terms['ELcgl_sdr'])
y2 = linear_op_from_list_term(coeffs,quantities,dic_of_list_terms['ELiso_sdr'])
splot(axe_perp[1:],np.nanmean((y1-y2)*filt_45binf,axis=1)[1:],label = '$\mathcal{S}_{A3}$') 

plt.xscale('log')
plt.yscale('log')
plt.xlim(axe_perp[1],axe_perp[-1])
plt.ylim(1e-11,1e-5)
#plt.ylim(1e-2,1e2)
plt.ylabel('CGL5')
plt.xlabel('$\ell_{\\perp}/d_i$')
plt.legend(ncol=3,loc='lower left',columnspacing=0.4,labelspacing = 0.2,borderpad=0.2)
plt.tight_layout()
plt.show()

In [15]:
X,Y = np.meshgrid(axe_perp,axe_par)
ELcgl = linear_op_from_list_term(coeffs,quantities,dic_of_list_terms['EL'])
ELiso = linear_op_from_list_term(coeffs,quantities,dic_of_list_terms['ELIso'])
Z = (ELcgl-ELiso)/np.abs(ELiso)*100

fig = plt.figure('M5', figsize=(12,10))
plt.clf()

spec = gridspec.GridSpec(ncols=3, nrows=2,
                         width_ratios=[0.7, 2,0.7], wspace=0,
                         hspace=0, height_ratios=[2, 0.7])


ax1 = plt.subplot(spec[1])
ax1.set_title('CGL5')
# colormap = ax1.pcolor(X,Y,np.transpose(Z), norm=colors.SymLogNorm(linthresh = 1e-8, vmin = - np.max(np.abs(Z)), vmax=np.max(np.abs(Z))),
#                    cmap='Spectral_r', shading='auto')

# contours = ax1.contour(X,Y,np.transpose(Z),10,cmap='Spectral',vmin = - np.max(Z), vmax=np.max(Z))
colormap = ax1.pcolor(X,Y,np.transpose(Z), norm=colors.SymLogNorm(linthresh = 1e0, vmin = - np.max(np.abs(Z))/10000, vmax=np.max(np.abs(Z))/10000),
                   cmap='Spectral_r', shading='auto')

contours = ax1.contour(X,Y,np.transpose(Z),[-100,-80,-60,-40,-20,-10,0,10,20,40,60,80,100],clim=(-100.0, 100.0),cmap='Spectral',vmin = -100, vmax=100)


ax1.plot(axe_perp[1:],axe_perp[1:]*np.tan(theta),'k-.',label='$\\theta = \\theta_i$')
ax1.set_xscale('log')
ax1.set_yscale('log')
ax1.set_xlim(axe_perp[1],axe_perp[-1])
ax1.set_ylim(axe_par[1],axe_par[-1])
ax1.xaxis.set_tick_params(labelbottom=False)

ax0 = plt.subplot(spec[0],sharey=ax1)
subsplot(ax0,axe_par[1:],np.nanmean(ELcgl*filt_45bsup,axis=0)[1:],label = '$\\varepsilon_{cgl}$',orientation = 'inverse') 
subsplot(ax0,axe_par[1:],np.nanmean(ELiso*filt_45bsup,axis=0)[1:],label = '$\\varepsilon_{iso}$',orientation = 'inverse') 
subsplot(ax0,axe_par[1:],np.nanmean((ELcgl-ELiso)*filt_45bsup,axis=0)[1:],label = '$\\varepsilon_{\overline{\\boldsymbol{\\Pi}}}$',orientation = 'inverse') 
subsplot(ax0,axe_par[1:],np.nanmean(np.abs(zeta)*filt_45bsup,axis=0)[1:],label = '$\\zeta$',color = 'grey',orientation = 'inverse')
plt.fill_betweenx(axe_par[1:],0,np.nanmean(np.abs(zeta)*filt_45bsup,axis=0)[1:],color = 'grey', alpha = 0.3)
ax0.plot([2,4],[4,5],'k-.',label='$\\theta = \\theta_i$')

ax0.set_xscale('log')
ax0.set_yscale('log')
ax0.set_xlim(2e-8,5e-5)
ax0.set_ylabel('$\ell_{\\parallel}/d_i$')

ax4 = plt.subplot(spec[4],sharex=ax1)
subsplot(ax4,axe_perp[1:],np.nanmean(ELcgl*filt_45binf,axis=1)[1:],label = '$\\varepsilon_{cgl}$') 
subsplot(ax4,axe_perp[1:],np.nanmean(ELiso*filt_45binf,axis=1)[1:],label = '$\\varepsilon_{iso}$') 
subsplot(ax4,axe_perp[1:],np.nanmean((ELcgl-ELiso)*filt_45binf,axis=1)[1:],label = '$\\varepsilon_{\overline{\\boldsymbol{\\Pi}}}$') 
subsplot(ax4,axe_perp[1:],np.nanmean(np.abs(zeta)*filt_45binf,axis=1)[1:],label = '$\\zeta$',color = 'grey')
plt.fill_between(axe_perp[1:],np.nanmean(np.abs(zeta)*filt_45binf,axis=1)[1:],color = 'grey', alpha = 0.3)
ax4.set_xscale('log')
ax4.set_yscale('log')
ax4.set_ylim(2e-8,5e-5)
ax4.set_xlabel('$\ell_{\\perp}/d_i$')

ax2 = plt.subplot(spec[2])
ax2.axis('off')



cbar = fig.colorbar(contours,ax = [ax2,],location='left',anchor=(0.5,0),pad=0.5, fraction=1,) #
cbar.formatter.set_powerlimits((0, 0))
cbar.update_ticks()
cbar2 = fig.colorbar(colormap,ax = [ax2,],location='right',label='$\\frac{\\varepsilon_{cgl} - \\varepsilon_{iso}}{|\\varepsilon_{iso}|}(\%)$',anchor=(-0.6,0), fraction=1) #

ax0.legend(bbox_to_anchor=(5.3, 0))
plt.show()

/tmp/ipykernel_12741/93502956.py:7: UserWarning: Attempted to set non-positive left xlim on a log-scaled axis.
Invalid limit will be ignored.
  plt.clf()
/tmp/ipykernel_12741/93502956.py:7: UserWarning: Attempted to set non-positive bottom ylim on a log-scaled axis.
Invalid limit will be ignored.
  plt.clf()
/tmp/ipykernel_12741/93502956.py:23: UserWarning: The following kwargs were not used by contour: 'clim'
  contours = ax1.contour(X,Y,np.transpose(Z),[-100,-80,-60,-40,-20,-10,0,10,20,40,60,80,100],clim=(-100.0, 100.0),cmap='Spectral',vmin = -100, vmax=100)
/tmp/ipykernel_12741/93502956.py:34: RuntimeWarning: Mean of empty slice
  subsplot(ax0,axe_par[1:],np.nanmean(ELcgl*filt_45bsup,axis=0)[1:],label = '$\\varepsilon_{cgl}$',orientation = 'inverse')
/tmp/ipykernel_12741/93502956.py:35: RuntimeWarning: Mean of empty slice
  subsplot(ax0,axe_par[1:],np.nanmean(ELiso*filt_45bsup,axis=0)[1:],label = '$\\varepsilon_{iso}$',orientation = 'inverse')
/tmp/ipykernel_12741/93502956.py:36: Ru